# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [21]:
# import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [50]:
#orders = zipfile.ZipFile
orders = pd.read_csv('Orders.zip').iloc[:,1:]
orders.head()

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [159]:
apcos = orders.groupby(['CustomerID']).agg({'amount_spent':sum})
iq95 = np.percentile(apcos['amount_spent'],95)
iq75 = np.percentile(apcos['amount_spent'],75)


VIP = apcos[apcos['amount_spent'] > iq95].reset_index()
Preferred = apcos[(apcos['amount_spent'] < iq75)&(apcos['amount_spent']<= iq95)].reset_index()

VIP_list = list(VIP['CustomerID'])
Preferred_list = list(Preferred['CustomerID'])

Status = []
for x in orders['CustomerID']:
    if x in VIP_list:
        s = 'VIP'
    elif x in Preferred_list:
        s = 'Preferred'
    else:
        s = 'Regular'
    Status.append(s)
orders['Status'] = Status

Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [161]:
#considering most VIP customers in % of total customers
VIP_ratio = orders.groupby(['Country','Status']).size()/orders.groupby(['Country']).size()*100
VIP_ratio = VIP_ratio.reset_index(name = 'Ratio')
VIP_ratio[VIP_ratio['Status'] == 'VIP'].sort_values(by = ['Ratio'], ascending = False)

,Country,Status,Ratio
60,Singapore,VIP,100.000000
23,EIRE,VIP,97.775629
49,Netherlands,VIP,88.023699
2,Australia,VIP,75.780591
43,Japan,VIP,63.862928
14,Channel Islands,VIP,48.663102
57,Portugal,VIP,46.580027
66,Sweden,VIP,43.902439
27,Finland,VIP,42.919708
17,Cyprus,VIP,40.390879


In [191]:
#considering most VIP customers (absolute value)
ncust = orders.groupby(['Country','Status']).size()
ncust = ncust.reset_index(name = 'N_customers')
ncust[ncust['Status'] == 'VIP'].sort_values(by = ['N_customers'], ascending = False)

,Country,Status,N_customers
74,United Kingdom,VIP,84185
23,EIRE,VIP,7077
30,France,VIP,3290
33,Germany,VIP,3127
49,Netherlands,VIP,2080
2,Australia,VIP,898
57,Portugal,VIP,681
69,Switzerland,VIP,594
63,Spain,VIP,511
52,Norway,VIP,420


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [195]:
VP = ncust[ncust['Status'] != 'Regular']
VP.groupby(['Country']).agg({'N_customers': sum}).sort_values(by = ['N_customers'], ascending = False)

,N_customers
Country,
United Kingdom,216895
EIRE,7077
France,5331
Germany,4820
Netherlands,2363
Spain,1427
Switzerland,1066
Australia,1055
Portugal,1050
